In [8]:
from math import ceil

import pandas as pd

from ddeserts import read_cvap_csv, moe_of_sum, moe_of_ratio

In [2]:
# read in list of charter cities
charter_cities_path = 'data/cacities/charter-cities.txt'

with open(charter_cities_path) as f:
    charter_cities = { line.strip() for line in f }

In [3]:
# load the data
places_path = 'data/census/CVAP_2015-2019_ACS_csv_files/Place.csv'

pre_filter = lambda line: 'California' in line and 'Total' in line
filter = lambda row: row['state'] == 'California' and row['geotype'] != 'cdp'

ca_city_rows = list(read_cvap_csv(places_path, pre_filter=pre_filter, filter=filter))


df = pd.DataFrame.from_records(ca_city_rows, index=['table', 'line'])
df['tot_moe'] = df['tot_moe'].astype('int')
df['adu_moe'] = df['adu_moe'].astype('int')
df['cvap_moe'] = df['cvap_moe'].astype('int')
df['cit_moe'] = df['cit_moe'].astype('int')


In [15]:
# add extra columns
df['has_charter'] = df['name'].isin(charter_cities)

df['dvap_est'] = df['adu_est'] - df['cvap_est']
df['dvap_moe'] = df.apply(lambda r: ceil(moe_of_sum(r['adu_moe'], r['cvap_moe'])), axis=1).astype('int')
df['dvap_pct'] = df['dvap_est'] / df['adu_est']
df['dvap_pct_moe'] = df.apply(lambda r: moe_of_ratio(
    r['dvap_est'], r['dvap_moe'], r['adu_est'], r['adu_moe']), axis=1).astype('float')


In [24]:
dvap = df.loc[df['has_charter']][
    ['name', 'adu_est', 'adu_moe', 'dvap_est', 'dvap_moe', 'dvap_pct', 'dvap_pct_moe']].sort_values('dvap_pct')
dvap[-30:]

name  adu_est  adu_moe  dvap_est  dvap_moe  \
table line                                                              
Place 27223          Buena Park    64245      807     12855      1368   
      29732  Desert Hot Springs    21440      700      4350      1173   
      31903            Glendale   164850      999     33510      2044   
      40860          San Rafael    45835      474      9335       938   
      30499           El Centro    31050      557      6325       936   
      33268           Inglewood    83810     1049     17150      1667   
      40665            San Jose   797090     1566    164515      4078   
      25247              Albany    14665      241      3060       590   
      40795           San Mateo    82505      817     17400      1420   
      32618             Hayward   125155      887     27275      1738   
      33346              Irvine   212955     1454     47620      3036   
      35400         Los Angeles  3145520     3613    728480      8499   
      39651        Redwood City    67430      749     15885      1422   
      39742            Richmond    85930      921     20620      1923   
      39105              Pomona   114115      947     27900      1724   
      25546             Anaheim   265820     1596     66135      3423   
      28900             Compton    68500      942     17275      1551   
      44630            Woodlake     4855      259      1355       439   
      29849              Dinuba    15740      414      4445       697   
      40925         Santa Clara   102390      787     29365      1797   
      37077       Mountain View    65625      660     19675      1403   
      41354             Shafter    12995      398      4220       713   
      43928         Watsonville    37495      622     12470      1204   
      40899           Santa Ana   243370     1502     83035      2932   
      26287                Bell    25885      610      8925       917   
      42407           Sunnyvale   120715      822     42505      1930   
      40990         Santa Maria    72750      872     26085      1627   
      40340             Salinas   107810      882     42475      1842   
      34984             Lindsay     8950      457      3665       674   
      33788           King City     9245      456      5415       701   

             dvap_pct  dvap_pct_moe  
table line                           
Place 27223  0.200093      0.024110  
      29732  0.202892      0.063405  
      31903  0.203276      0.013714  
      40860  0.203665      0.022807  
      30499  0.203704      0.034417  
      33268  0.204630      0.022736  
      40665  0.206395      0.005532  
      25247  0.208660      0.044390  
      40795  0.210896      0.019492  
      32618  0.217930      0.015541  
      33346  0.223615      0.015892  
      35400  0.231593      0.002971  
      39651  0.235578      0.023972  
      39742  0.239963      0.025221  
      39105  0.244490      0.017280  
      25546  0.248796      0.014458  
      28900  0.252190      0.026474  
      44630  0.279094      0.111246  
      29849  0.282402      0.053107  
      40925  0.286796      0.019908  
      37077  0.299810      0.024642  
      41354  0.324740      0.066861  
      43928  0.332578      0.038263  
      40899  0.341188      0.014241  
      26287  0.344794      0.044602  
      42407  0.352110      0.018512  
      40990  0.358557      0.026985  
      40340  0.393980      0.020476  
      34984  0.409497      0.101394  
      33788  0.585722      0.110148

In [19]:
dvap = df.loc[df['has_charter']][
    ['name', 'adu_est', 'adu_moe', 'dvap_est', 'dvap_moe', 'dvap_pct', 'dvap_pct_moe']].sort_values('dvap_est')
dvap[-30:]

name  adu_est  adu_moe  dvap_est  dvap_moe  dvap_pct  \
table line                                                                    
Place 28900         Compton    68500      942     17275      1551  0.252190   
      40795       San Mateo    82505      817     17400      1420  0.210896   
      36583         Modesto   158385     1042     17825      1798  0.112542   
      37077   Mountain View    65625      660     19675      1403  0.299810   
      38442        Pasadena   115505      864     19685      1731  0.170426   
      41042      Santa Rosa   140870      836     20410      1779  0.144885   
      39742        Richmond    85930      921     20620      1923  0.239963   
      40990     Santa Maria    72750      872     26085      1627  0.358557   
      32618         Hayward   125155      887     27275      1738  0.217930   
      39105          Pomona   114115      947     27900      1724  0.244490   
      40457  San Bernardino   152070     1556     28590      2558  0.188006   
      40925     Santa Clara   102390      787     29365      1797  0.286796   
      31903        Glendale   164850      999     33510      2044  0.203276   
      39937       Riverside   248645     1714     35775      2635  0.143880   
      26040     Bakersfield   264830     1753     36715      2963  0.138636   
      42199        Stockton   223645     1052     36910      2403  0.165038   
      28432     Chula Vista   200870     1391     37005      2888  0.184224   
      40340         Salinas   107810      882     42475      1842  0.393980   
      42407       Sunnyvale   120715      822     42505      1930  0.352110   
      40301      Sacramento   385200     1454     44720      2563  0.116096   
      33346          Irvine   212955     1454     47620      3036  0.223615   
      31617          Fresno   375795     1548     52895      3111  0.140755   
      35309      Long Beach   362340     1852     54060      3234  0.149197   
      37740         Oakland   340625     1274     55755      2591  0.163684   
      25546         Anaheim   265820     1596     66135      3423  0.248796   
      40899       Santa Ana   243370     1502     83035      2932  0.341188   
      40587   San Francisco   757415        0    104965      2317  0.138583   
      40535       San Diego  1129095     2591    157595      5312  0.139576   
      40665        San Jose   797090     1566    164515      4078  0.206395   
      35400     Los Angeles  3145520     3613    728480      8499  0.231593   

             dvap_pct_moe  
table line                 
Place 28900      0.026474  
      40795      0.019492  
      36583      0.012173  
      37077      0.024642  
      38442      0.016384  
      41042      0.013569  
      39742      0.025221  
      40990      0.026985  
      32618      0.015541  
      39105      0.017280  
      40457      0.018939  
      40925      0.019908  
      31903      0.013714  
      39937      0.011670  
      26040      0.012187  
      42199      0.011575  
      28432      0.015762  
      40340      0.020476  
      42407      0.018512  
      40301      0.007119  
      33346      0.015892  
      31617      0.008895  
      35309      0.009738  
      37740      0.008250  
      25546      0.014458  
      40899      0.014241  
      40587      0.003059  
      40535      0.005037  
      40665      0.005532  
      35400      0.002971

name  adu_est  adu_moe  dvap_est  dvap_moe  dvap_pct  \
table line                                                                    
Place 31786    Garden Grove   136210      983     24430      1858  0.179355   
      36908   Moreno Valley   148980     1352     24600      2324  0.165123   
      37961         Ontario   130330     1199     25745      2336  0.197537   
      40990     Santa Maria    72750      872     26085      1627  0.358557   
      32618         Hayward   125155      887     27275      1738  0.217930   
      39105          Pomona   114115      947     27900      1724  0.244490   
      31266         Fontana   150425     1480     28040      2380  0.186405   
      30642        El Monte    88970      957     28440      1917  0.319658   
      40457  San Bernardino   152070     1556     28590      2558  0.188006   
      40925     Santa Clara   102390      787     29365      1797  0.286796   
      31903        Glendale   164850      999     33510      2044  0.203276   
      39937       Riverside   248645     1714     35775      2635  0.143880   
      26040     Bakersfield   264830     1753     36715      2963  0.138636   
      42199        Stockton   223645     1052     36910      2403  0.165038   
      28432     Chula Vista   200870     1391     37005      2888  0.184224   
      38117          Oxnard   151640      939     39800      2024  0.262464   
      40340         Salinas   107810      882     42475      1842  0.393980   
      42407       Sunnyvale   120715      822     42505      1930  0.352110   
      40301      Sacramento   385200     1454     44720      2563  0.116096   
      31565         Fremont   181815     1021     46910      1991  0.258010   
      33346          Irvine   212955     1454     47620      3036  0.223615   
      31617          Fresno   375795     1548     52895      3111  0.140755   
      35309      Long Beach   362340     1852     54060      3234  0.149197   
      37740         Oakland   340625     1274     55755      2591  0.163684   
      25546         Anaheim   265820     1596     66135      3423  0.248796   
      40899       Santa Ana   243370     1502     83035      2932  0.341188   
      40587   San Francisco   757415        0    104965      2317  0.138583   
      40535       San Diego  1129095     2591    157595      5312  0.139576   
      40665        San Jose   797090     1566    164515      4078  0.206395   
      35400     Los Angeles  3145520     3613    728480      8499  0.231593   

             dvap_pct_moe  
table line                 
Place 31786      0.015044  
      36908      0.017254  
      37961      0.019924  
      40990      0.026985  
      32618      0.015541  
      39105      0.017280  
      31266      0.017831  
      30642      0.025257  
      40457      0.018939  
      40925      0.019908  
      31903      0.013714  
      39937      0.011670  
      26040      0.012187  
      42199      0.011575  
      28432      0.015762  
      38117      0.015066  
      40340      0.020476  
      42407      0.018512  
      40301      0.007119  
      31565      0.012470  
      33346      0.015892  
      31617      0.008895  
      35309      0.009738  
      37740      0.008250  
      25546      0.014458  
      40899      0.014241  
      40587      0.003059  
      40535      0.005037  
      40665      0.005532  
      35400      0.002971

In [29]:
#df.to_csv('output/ca_cities.csv')